In [12]:
# Import necessary libraries
import openai
import mysql.connector
import psycopg2
import sqlite3
import time
from langchain_openai import OpenAI  # Import the updated OpenAI class
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI as LLMOpenAI  # New OpenAI import
from langchain.vectorstores import FAISS

In [13]:
#Set up OpenAI GPT-4 API key
openai.api_key = "your_api_key"

In [14]:
#Function to dynamically connect to different types of databases
def get_db_connection(host, user, password, db_name):
    """
    Connects to the database with the provided connection details.

    :param host: Host of the database
    :param user: Username for the database
    :param password: Password for the database
    :param db_name: Database name
    :return: Database connection object
    """
    return mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=db_name
    )

In [15]:
#Function to get schema information from the database
def get_schema_info(db_connection):
    """
    Retrieves the schema information from the connected database.

    :param db_connection: The database connection object
    :return: A dictionary containing tables and their columns
    """
    cursor = db_connection.cursor()

    # Get the list of tables (generic for most SQL databases)
    cursor.execute("SHOW TABLES;")
    tables = cursor.fetchall()

    schema_info = {}

    for table in tables:
        table_name = table[0]

        # Get the list of columns for each table
        cursor.execute(f"DESCRIBE {table_name};")
        columns = cursor.fetchall()

        # Add table and its columns to schema info
        schema_info[table_name] = [column[0] for column in columns]

    cursor.close()
    return schema_info

In [16]:
#LangChain prompt template for SQL query generation
sql_prompt = """
You are a SQL expert. You will generate SQL queries from natural language questions.

Database Schema: {schema_info}

Question: {question}

SQL Query:
"""
prompt_template = PromptTemplate(input_variables=["schema_info", "question"], template=sql_prompt)
llm = LLMOpenAI(temperature=0, openai_api_key=openai.api_key)  # Using the new OpenAI class
chain = prompt_template | llm  # Updated chain creation with RunnableSequence

In [17]:
#Function to run the generated SQL query on a given database
def run_sql_query(db_connection, query):
    """
    Executes the generated SQL query on the provided database.

    :param db_connection: The database connection object
    :param query: The SQL query to execute
    :return: The result of the query
    """
    cursor = db_connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    return result

In [ ]:
#Main function to interact with the user (using hardcoded values for database details)
def main():
    """
    Main function to interact with the user and generate SQL queries from natural language.
    Uses hardcoded database credentials and example question.
    """
    # Directly providing the database details
    db_type = 'mysql'  # Use the correct DB type ('mysql', 'postgresql', 'sqlite')
    host = 'localhost'  # Your database host (use 'localhost' or IP for MySQL)
    user = 'root'       # Your MySQL user
    password = 'password' # Your MySQL password
    db_name = 'loan_data'  # Your database name
    
    # Connect to the database
    db_connection = get_db_connection(host, user, password, db_name)

    # Discover the schema
    schema_info = get_schema_info(db_connection)
    print(f"Discovered Schema: {schema_info}")

    # Loop to ask for queries
    while True:
        # Hardcoded question for now
        question = "How many loans were approved?"  # Example question

        # Measure time for LangChain SQL query generation
        start_time = time.time()

        # Generate SQL query using LangChain
        sql_query = chain.invoke({"schema_info": schema_info, "question": question})  # Use invoke()

        end_time = time.time()
        generation_time = end_time - start_time
        print(f"Time taken to generate SQL query: {generation_time} seconds")

        print(f"Generated SQL Query: {sql_query}")

        # Execute the SQL query on the database
        try:
            result = run_sql_query(db_connection, sql_query)
            print(f"Query Result: {result}")
        except Exception as e:
            print(f"Error executing query: {e}")

# Run the main function
if __name__ == "__main__":
    main()

Discovered Schema: {'loan': ['Loan_ID', 'Loan_Amount', 'Credit_Score', 'Income', 'Employment_Type', 'Loan_Term', 'Past_Defaults', 'Loan_Status']}


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}